In [1]:

%reset
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import random
import treecorr
from astropy.io import fits
import pickle as pk
import os.path
from os import path
import mycosmo as cosmodef
import scipy as sp
import scipy.interpolate as interpolate
from scipy.integrate import quad
from scipy.optimize import fsolve
import scipy.optimize as op



Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:

class general_funcs:

    def __init__(self, cosmo_params):
        h = cosmo_params['H0'] / 100.
        cosmo_func = cosmodef.mynew_cosmo(h, cosmo_params['Om0'], cosmo_params['Ob0'], cosmo_params['ns'],
                                          cosmo_params['sigma8'])
        self.cosmo = cosmo_func

    def get_Dcom(self, zf):
        c = 3 * 10 ** 5
        Omega_m, Omega_L = self.cosmo.Om0, 1. - self.cosmo.Om0
        res1 = sp.integrate.quad(lambda z: (c / 100) * (1 / (np.sqrt(Omega_L + Omega_m * ((1 + z) ** 3)))), 0, zf)
        Dcom = res1[0]
        return Dcom

    def get_Dcom_array(self,zarray):
        Omega_m = self.cosmo.Om0
        Omega_L = 1. - Omega_m
        c = 3 * 10 ** 5
        Dcom_array = np.zeros(len(zarray))
        for j in range(len(zarray)):
            zf = zarray[j]
            res1 = sp.integrate.quad(lambda z: (c / 100) * (1 / (np.sqrt(Omega_L + Omega_m * ((1 + z) ** 3)))), 0, zf)
            Dcom = res1[0]
            Dcom_array[j] = Dcom
        return Dcom_array

    def get_Hz(self,zarray):
        Omega_m = self.cosmo.Om0
        Omega_L = 1 - Omega_m
        Ez = np.sqrt(Omega_m * (1 + zarray) ** 3 + Omega_L)
        Hz = 100. * Ez
        return Hz

    def get_diff(self, zf, chi):
        return chi - self.get_Dcom(zf)

    def root_find(self, init_x, chi):
        nll = lambda *args: self.get_diff(*args)
        result = op.root(nll, np.array([init_x]), args=chi, options={'maxfev': 50}, tol=0.01)
        return result.x[0]

    def get_z_from_chi(self, chi):
        valf = self.root_find(0., chi)
        return valf
    
    
cosmo_params_dict = {'flat': True, 'H0': 70.0, 'Om0': 0.25, 'Ob0': 0.044, 'sigma8': 0.8, 'ns': 0.95}
gnf = general_funcs(cosmo_params_dict)
z_array = np.linspace(0, 1.5, 10000)
chi_array = np.zeros(len(z_array))
for j in range(len(z_array)):
    chi_array[j] = gnf.get_Dcom(z_array[j])
chi_interp = interpolate.interp1d(z_array, chi_array)


chi_array = np.linspace(0, 4000, 50000)
z_array = np.zeros(len(chi_array))
for j in range(len(z_array)):
    z_array[j] = gnf.get_z_from_chi(chi_array[j])
z_interp = interpolate.interp1d(chi_array, z_array)





In [22]:
halo_new = fits.open('/global/project/projectdirs/des/shivamp/actxdes/data_set/mice_sims/6144.fits')
halo_new[1].header



XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   24 / length of dimension 1                          
NAXIS2  =            155554576 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    5 / number of table fields                         
TTYPE1  = 'unique_halo_id'                                                      
TFORM1  = 'K       '                                                            
TTYPE2  = 'xhalo   '                                                            
TFORM2  = 'E       '                                                            
TTYPE3  = 'zhalo   '        

In [25]:
halo_id_all, xhalo_all, yhalo_all, zhalo_all = (halo_new[1].data['unique_halo_id']), (halo_new[1].data['xhalo']), (halo_new[1].data['yhalo']), (halo_new[1].data['zhalo'])

halo_id_unique, ind_unique = np.unique(halo_id_all, return_index=True)

len(halo_id_unique), len(halo_id_all)


(40674733, 155554576)

In [34]:
lm_halo_all = (halo_new[1].data['lmhalo'])
lm_halo_uid = lm_halo_all[ind_unique]

np.min(lm_halo_uid), np.max(lm_halo_uid)



(12.0001, 15.2683)

In [69]:
xhalo_uid, yhalo_uid, zhalo_uid = xhalo_all[ind_unique], yhalo_all[ind_unique], zhalo_all[ind_unique]

ind_nz = np.where( (xhalo_uid > 0) & (yhalo_uid > 0) & (zhalo_uid > 0)  )[0]

xhalo_uid, yhalo_uid, zhalo_uid, lm_halo_uid = xhalo_uid[ind_nz], yhalo_uid[ind_nz], zhalo_uid[ind_nz], lm_halo_uid[ind_nz]



In [33]:

ind_msel = np.where( (lm_halo > 14.5) & (lm_halo < 15.0)  )[0]

len(ind_msel)


2312

In [88]:
ds = 32
ind_rand = np.unique(np.random.randint(0, len(xhalo_uid), int(len(xhalo_uid)/ds)))
print(len(ind_rand))

if ds == 1:
    xhalo, yhalo, zhalo, lm_halo = xhalo_uid, yhalo_uid, zhalo_uid, lm_halo_uid 
else:
    xhalo, yhalo, zhalo, lm_halo = xhalo_uid[ind_rand], yhalo_uid[ind_rand], zhalo_uid[ind_rand], lm_halo_uid[ind_rand]


ra_h = (180/np.pi)*np.arctan(xhalo/yhalo) 
dec_h= 90.-(180/np.pi)*np.arctan(np.sqrt(xhalo**2 + yhalo**2)/zhalo)
chi_h = np.sqrt(xhalo**2 + yhalo**2 + zhalo**2)
z_h = z_interp(chi_h)




1251095


In [89]:
save_dir= '/global/project/projectdirs/des/shivamp/actxdes/data_set/mice_sims/'


save_filename = 'MICEv2_halos_Mlow_1e12_ds_' + str(ds) + '.fits'
    
c1 = fits.Column(name='ra', array=ra_h, format='E')
c2 = fits.Column(name='dec', array=dec_h, format='E')
c3 = fits.Column(name='z', array=z_h, format='E')
c4 = fits.Column(name='log_m', format='E', array=lm_halo)
t = fits.BinTableHDU.from_columns([c1, c2, c3, c4])
t.writeto(save_dir + save_filename, clobber=True)



In [73]:
len(ra_h)



40674723

In [ ]:
ra_h = (180/np.pi)*np.arctan(xg/yg) 
dec_h= 90.-(180/np.pi)*np.arctan(np.sqrt(xg**2 + yg**2)/zg)
chi_h = np.sqrt(xg**2 + yg**2 + zg**2)
zc_s = z_interp(chi_s)



In [13]:
xg, yg, zg = halo_new[1].data['xgal'],halo_new[1].data['ygal'],halo_new[1].data['zgal']

ra_g, dec_g, zc_gal = halo_new[1].data['ra_gal'],halo_new[1].data['dec_gal'],halo_new[1].data['z_cgal']





In [15]:
ra_s = (180/np.pi)*np.arctan(xg/yg) 
dec_s= 90.-(180/np.pi)*np.arctan(np.sqrt(xg**2 + yg**2)/zg)
chi_s = np.sqrt(xg**2 + yg**2 + zg**2)
zc_s = z_interp(chi_s)



/global/u1/s/spandey/myenv_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in divide
  """Entry point for launching an IPython kernel.
/global/u1/s/spandey/myenv_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in arctan
  """Entry point for launching an IPython kernel.
/global/u1/s/spandey/myenv_new/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in divide
  
/global/u1/s/spandey/myenv_new/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in arctan
  


In [11]:
ra_s - ra_g



array([ 3.48639587e-04, -4.06372070e-07,  3.63418579e-07, ...,
        1.46278687e-04,  3.86387451e-04,  3.41129883e-04])

In [9]:
dec_s - dec_g




array([ 1.74187012e-04, -1.85351563e-06, -1.15405274e-06, ...,
       -1.12997803e-04, -3.13363281e-04,  2.89309082e-05])

In [16]:
zc_s - zc_gal



array([-0.0002286 , -0.00015826, -0.00021178, ..., -0.00029645,
       -0.00013817, -0.00015506])